In [4]:
import random
import cv2
import math
import imageio
from skimage.measure import compare_ssim as ssim

MAX_NUMBER_OF_FRAMES = 497

TOTAL_KEY_FRAMES = 10

STOPPING_ITERATION = 10

NUMBER_OF_NP_CANDIDATES = 10

# Location to read images from.
# NOTE: "_" is used to follow a naming convention
# eg. _20.jpg, _21.jpg etc...
READ_LOCATION = "C:/Users/Brian/Desktop/Desktop/Thesis/IMDb_Scrape/Movie_Trailers/Deadpool_2/frames/_"

# Location to write GIF images to.
GIF_WRITE_LOCATION = "C:/Users/Brian/Desktop/Desktop/Thesis/IMDb_Scrape/Movie_Trailers/Deadpool_2/key_frames/avg_SSIM.GIF"

# Population matrix.
NP = []

# Mutation vector.
MV = []

# Trail vector.
TV = []

# Scale factor.
F = 0.9

# Cr probability value.
Cr = 0.6

# Calculate ASSIM for a chromosome.
def getASSIM( KF ):
    ssim_sum = 0
    for i in range(1, TOTAL_KEY_FRAMES - 1):
        while True:
            try:
                im1 = cv2.imread(READ_LOCATION + str(KF[i]) + ".jpg",0)
                im2 = cv2.imread(READ_LOCATION + str(KF[i+1]) + ".jpg",0)
                ssim_sum += ssim(im1, im2)
            except:
                print(i, KF, KF[i], KF[i+1])
                continue
            break
    return ssim_sum/(TOTAL_KEY_FRAMES - 1)

# INITIALISATION : Generates population NP of 10 parent vectors (and ASSIMs).
def initialize_NP():
    for i in range(NUMBER_OF_NP_CANDIDATES):
        NP.append(sorted(random.sample(range(1, MAX_NUMBER_OF_FRAMES+1), TOTAL_KEY_FRAMES)))
        NP[-1].append(getASSIM(NP[-1]))
        print(NP[-1])

# MUTATION
def mutation(parent):
    R = random.sample(NP,2)
    global MV
    MV[:] = []
    MV_value = 0
    print(NP[parent])
    for i in range(TOTAL_KEY_FRAMES):
        MV_value = int(NP[parent][i] + F*(R[0][i] - R[1][i]))
        if(MV_value < 1):
            MV.append(1)
        elif(MV_value > MAX_NUMBER_OF_FRAMES):
            MV.append(MAX_NUMBER_OF_FRAMES)
        else:
            MV.append(MV_value)
    MV.sort()
    MV.append(getASSIM(MV))

# CROSSOVER (uniform crossover with Cr = 0.6).
def crossover(parent, mutant):
    print("mutant: ", mutant)
    print("parent: ", parent)
    for j in range(TOTAL_KEY_FRAMES) :
        if(random.uniform(0,1) < Cr) :
            TV.append(mutant[j])
        else:
            TV.append(parent[j])
    TV.sort()
    TV.append(getASSIM(TV))
    print("TV    : ", TV)

# SELECTION : Selects offspring / parent based on lower ASSIM value.
def selection(parent, trail_vector):
    if(trail_vector[-1] < parent[-1]):
        parent[:] = trail_vector
        print("yes", parent)
    else:
        print("no")

# bestParent returns the parent with then minimum ASSIM value.
def bestParent(population):
    Min_ASSIM_value = population[0][-1]
    Best_Parent_Index = population[0]
    for parent in population:
        if (parent[-1] < Min_ASSIM_value):
            Min_ASSIM_value = parent[-1]
            Best_Parent_Index = parent
    return Best_Parent_Index




initialize_NP()
for GENERATION in range(STOPPING_ITERATION):
    for i in range(NUMBER_OF_NP_CANDIDATES):
        print("---------------------", "PARENT:", i+1 , "GENERATION:", GENERATION+1, "---------------------")
        mutation(i)
        crossover(NP[i], MV)
        selection(NP[i], TV)
        print(NP[i])
        TV[:] = []
        print("")
    print("")
best_parent = bestParent(NP)
print("best solution is: ", best_parent)
images_for_gif = []
for frame_number in best_parent[:-1]:
        images_for_gif.append(imageio.imread(READ_LOCATION + str(frame_number) + '.jpg'))
imageio.mimsave(GIF_WRITE_LOCATION, images_for_gif)

c:\users\brian\anaconda3\lib\site-packages\skimage\util\arraycrop.py:177: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  cropped = ar[slices]


[37, 75, 184, 239, 242, 339, 388, 440, 459, 483, 0.5176814846432252]
[51, 82, 93, 144, 178, 259, 365, 401, 434, 487, 0.6211666795783253]
[40, 55, 74, 76, 135, 272, 350, 406, 454, 482, 0.584326573290727]
[35, 52, 63, 116, 124, 193, 194, 223, 322, 471, 0.5331937784674227]
[26, 60, 168, 189, 227, 265, 266, 417, 469, 489, 0.49327791367937884]
[4, 144, 234, 238, 255, 314, 448, 455, 474, 496, 0.49978632603582934]
[11, 49, 70, 141, 151, 181, 224, 301, 384, 398, 0.5866925644677593]
[17, 110, 159, 318, 321, 414, 454, 457, 472, 487, 0.5325012210066444]
[21, 49, 99, 201, 238, 325, 335, 374, 430, 461, 0.5411640061111883]
[57, 60, 87, 100, 164, 265, 391, 476, 479, 487, 0.5735711563214689]
--------------------- PARENT: 1 GENERATION: 1 ---------------------
[37, 75, 184, 239, 242, 339, 388, 440, 459, 483, 0.5176814846432252]
mutant:  [13, 51, 81, 150, 160, 196, 240, 314, 391, 406, 0.6044109857492143]
parent:  [37, 75, 184, 239, 242, 339, 388, 440, 459, 483, 0.5176814846432252]
TV    :  [13, 51, 150, 

mutant:  [1, 17, 74, 89, 97, 306, 379, 419, 461, 497, 0.4913238384815448]
parent:  [11, 49, 70, 141, 151, 379, 384, 412, 458, 497, 0.530442589560218]
TV    :  [11, 17, 74, 89, 97, 306, 379, 419, 461, 497, 0.4913238384815448]
yes [11, 17, 74, 89, 97, 306, 379, 419, 461, 497, 0.4913238384815448]
[11, 17, 74, 89, 97, 306, 379, 419, 461, 497, 0.4913238384815448]

--------------------- PARENT: 8 GENERATION: 2 ---------------------
[17, 110, 159, 318, 321, 414, 454, 457, 472, 487, 0.5325012210066444]
mutant:  [40, 132, 158, 327, 356, 417, 462, 474, 475, 475, 0.650331397969286]
parent:  [17, 110, 159, 318, 321, 414, 454, 457, 472, 487, 0.5325012210066444]
TV    :  [40, 110, 159, 318, 321, 417, 454, 474, 475, 487, 0.6099778404944467]
no
[17, 110, 159, 318, 321, 414, 454, 457, 472, 487, 0.5325012210066444]

--------------------- PARENT: 9 GENERATION: 2 ---------------------
[18, 32, 49, 273, 297, 340, 375, 415, 428, 461, 0.5303112043195974]
mutant:  [6, 62, 88, 172, 218, 250, 369, 413, 429, 469

mutant:  [1, 1, 77, 139, 211, 265, 268, 360, 456, 461, 0.5631440216787101]
parent:  [26, 36, 91, 154, 223, 227, 252, 357, 442, 469, 0.43949208296971204]
TV    :  [1, 26, 77, 154, 211, 265, 268, 360, 456, 461, 0.5718138173180932]
no
[26, 36, 91, 154, 223, 227, 252, 357, 442, 469, 0.43949208296971204]

--------------------- PARENT: 6 GENERATION: 4 ---------------------
[4, 144, 203, 206, 234, 314, 448, 455, 474, 496, 0.41093647499778146]
mutant:  [4, 39, 87, 97, 150, 276, 354, 409, 456, 490, 0.5369893078513769]
parent:  [4, 144, 203, 206, 234, 314, 448, 455, 474, 496, 0.41093647499778146]
TV    :  [4, 87, 144, 206, 234, 276, 354, 409, 456, 496, 0.49432239184014776]
no
[4, 144, 203, 206, 234, 314, 448, 455, 474, 496, 0.41093647499778146]

--------------------- PARENT: 7 GENERATION: 4 ---------------------
[11, 17, 74, 89, 97, 306, 379, 419, 461, 497, 0.4913238384815448]
mutant:  [1, 71, 166, 170, 177, 355, 436, 449, 488, 496, 0.5235104193556374]
parent:  [11, 17, 74, 89, 97, 306, 379, 419

mutant:  [39, 42, 53, 84, 201, 235, 356, 467, 481, 487, 0.6396678428875093]
parent:  [1, 1, 42, 75, 141, 272, 346, 436, 451, 490, 0.5023855338207684]
TV    :  [1, 39, 42, 84, 201, 272, 356, 436, 481, 490, 0.5670889308796228]
no
[1, 1, 42, 75, 141, 272, 346, 436, 451, 490, 0.5023855338207684]

--------------------- PARENT: 4 GENERATION: 6 ---------------------
[35, 52, 63, 116, 124, 193, 194, 223, 322, 471, 0.5331937784674227]
mutant:  [1, 1, 5, 20, 35, 156, 175, 219, 294, 469, 0.5393610627046732]
parent:  [35, 52, 63, 116, 124, 193, 194, 223, 322, 471, 0.5331937784674227]
TV    :  [1, 20, 52, 63, 124, 156, 194, 219, 294, 469, 0.5714858270865713]
no
[35, 52, 63, 116, 124, 193, 194, 223, 322, 471, 0.5331937784674227]

--------------------- PARENT: 5 GENERATION: 6 ---------------------
[26, 36, 91, 154, 223, 227, 252, 357, 442, 469, 0.43949208296971204]
mutant:  [31, 270, 348, 349, 369, 427, 446, 469, 474, 497, 0.6307907352275344]
parent:  [26, 36, 91, 154, 223, 227, 252, 357, 442, 469, 0

TV    :  [64, 169, 176, 181, 279, 291, 409, 440, 474, 497, 0.463262194866907]
no
[1, 96, 176, 181, 273, 291, 388, 440, 482, 492, 0.4459682464382147]

--------------------- PARENT: 2 GENERATION: 8 ---------------------
[72, 83, 88, 116, 157, 259, 370, 436, 443, 497, 0.4606563819176171]
mutant:  [1, 1, 1, 1, 45, 76, 196, 365, 420, 497, 0.6207414091960994]
parent:  [72, 83, 88, 116, 157, 259, 370, 436, 443, 497, 0.4606563819176171]
TV    :  [1, 1, 1, 76, 116, 157, 196, 365, 443, 497, 0.46834634920464]
no
[72, 83, 88, 116, 157, 259, 370, 436, 443, 497, 0.4606563819176171]

--------------------- PARENT: 3 GENERATION: 8 ---------------------
[1, 1, 42, 75, 141, 272, 346, 436, 451, 490, 0.5023855338207684]
mutant:  [1, 1, 20, 120, 146, 199, 207, 233, 302, 473, 0.4615890903407593]
parent:  [1, 1, 42, 75, 141, 272, 346, 436, 451, 490, 0.5023855338207684]
TV    :  [1, 1, 42, 120, 141, 199, 233, 346, 451, 490, 0.49797790571387]
yes [1, 1, 42, 120, 141, 199, 233, 346, 451, 490, 0.49797790571387]
[

mutant:  [1, 10, 38, 81, 102, 332, 460, 497, 497, 497, 0.6112441954346876]
parent:  [57, 60, 87, 100, 128, 285, 445, 471, 487, 489, 0.5217106242832216]
TV    :  [1, 38, 60, 100, 128, 332, 460, 471, 497, 497, 0.6188576688552797]
no
[57, 60, 87, 100, 128, 285, 445, 471, 487, 489, 0.5217106242832216]


--------------------- PARENT: 1 GENERATION: 10 ---------------------
[1, 96, 176, 181, 273, 291, 388, 440, 482, 492, 0.4459682464382147]
mutant:  [1, 41, 64, 109, 195, 212, 261, 353, 487, 497, 0.5384497033916573]
parent:  [1, 96, 176, 181, 273, 291, 388, 440, 482, 492, 0.4459682464382147]
TV    :  [1, 96, 109, 176, 195, 212, 353, 388, 482, 497, 0.5665438588185461]
no
[1, 96, 176, 181, 273, 291, 388, 440, 482, 492, 0.4459682464382147]

--------------------- PARENT: 2 GENERATION: 10 ---------------------
[72, 83, 88, 116, 157, 259, 370, 436, 443, 497, 0.4606563819176171]
mutant:  [76, 89, 91, 121, 144, 174, 259, 293, 346, 481, 0.5041485573388746]
parent:  [72, 83, 88, 116, 157, 259, 370, 436,